In [11]:
import numpy as np
import pandas as pd
from datetime import datetime

# Функция для генерации данных с экспоненциальным распределением для purchase_amount
def generate_ab_test_data(n_control, n_experiment, start_date, end_date, control_conversion_rate, experiment_conversion_rate, control_arpu_mean, experiment_arpu_mean):
    # Список дней эксперимента
    date_range = pd.date_range(start=start_date, end=end_date, freq='h')
    
    # Генерация дат посещений
    def generate_session_dates(n_sessions):
        return np.random.choice(date_range, size=n_sessions, replace=True)

    # Вероятностные распределения
    product_views_dist = np.random.poisson(5, size=n_control + n_experiment)
    cart_adds_dist = np.random.poisson(2, size=n_control + n_experiment)
    session_duration_dist = np.random.exponential(scale=10, size=n_control + n_experiment)  # Экспоненциальное распределение для времени сессий
    
    # Распределение для device_type, traffic_source, region, visitor_type
    device_type_dist = np.random.choice(['mobile', 'desktop', 'tablet'], p=[0.7, 0.25, 0.05], size=n_control + n_experiment)
    traffic_source_dist = np.random.choice(['organic', 'paid_ad', 'direct'], p=[0.5, 0.3, 0.2], size=n_control + n_experiment)
    region_dist = np.random.choice(['Estonia', 'Latvia', 'Lithuania'], p=[0.3, 0.4, 0.3], size=n_control + n_experiment)
    
    # Добавляем распределение visitor_type (например, 60% старые, 40% новые)
    visitor_type_dist = np.random.choice(['new', 'old'], p=[0.4, 0.6], size=n_control + n_experiment)

    # Генерация данных для контрольной группы
    control_data = {
        'user_id': np.arange(1, n_control + 1),
        'group': ['control'] * n_control,
        'session_date': generate_session_dates(n_control),
        'product_views': product_views_dist[:n_control],
        'cart_adds': cart_adds_dist[:n_control],
        'purchase_amount': np.where(np.random.rand(n_control) < control_conversion_rate,  # Вероятность конверсии для контрольной группы
                                    np.random.exponential(scale=control_arpu_mean / 2, size=n_control), 0),  # Экспоненциальное распределение ARPU
        'session_duration': session_duration_dist[:n_control],
        'device_type': device_type_dist[:n_control],
        'traffic_source': traffic_source_dist[:n_control],
        'region': region_dist[:n_control],
        'visitor_type': visitor_type_dist[:n_control]  # Новый или старый пользователь
    }

    # Генерация данных для экспериментальной группы
    experiment_data = {
        'user_id': np.arange(n_control + 1, n_control + n_experiment + 1),
        'group': ['experiment'] * n_experiment,
        'session_date': generate_session_dates(n_experiment),
        'product_views': product_views_dist[n_control:],
        'cart_adds': cart_adds_dist[n_control:],
        'purchase_amount': np.where(np.random.rand(n_experiment) < experiment_conversion_rate,  # Вероятность конверсии для экспериментальной группы
                                    np.random.exponential(scale=experiment_arpu_mean / 2, size=n_experiment), 0),  # Экспоненциальное распределение ARPU
        'session_duration': session_duration_dist[n_control:],
        'device_type': device_type_dist[n_control:],
        'traffic_source': traffic_source_dist[n_control:],
        'region': region_dist[n_control:],
        'visitor_type': visitor_type_dist[n_control:]  # Новый или старый пользователь
    }

    # Преобразуем в DataFrame
    df_control = pd.DataFrame(control_data)
    df_experiment = pd.DataFrame(experiment_data)
    
    # Объединяем контрольную и экспериментальную группы
    df = pd.concat([df_control, df_experiment], ignore_index=True)
    
    # Сохраняем датасет
    df.to_csv('rimi_ab_test_data_exponential_with_visitors.csv', index=False)

    return df

# Настраиваем параметры для контрольной и экспериментальной групп
n_control = 4361
n_experiment = 4361
start_date = '2024-08-05'
end_date = '2024-08-14'
control_conversion_rate = 0.05  # 5% вероятность конверсии для контрольной группы
experiment_conversion_rate = 0.07  # 7% вероятность конверсии для экспериментальной группы
control_arpu_mean = 50  # Средний ARPU для контрольной группы
experiment_arpu_mean = 55  # Средний ARPU для экспериментальной группы

# Генерация данных
df = generate_ab_test_data(n_control, n_experiment, start_date, end_date, control_conversion_rate, experiment_conversion_rate, control_arpu_mean, experiment_arpu_mean)

# Пример первых строк датасета
df.head()


,user_id,group,session_date,product_views,cart_adds,purchase_amount,session_duration,device_type,traffic_source,region,visitor_type
0,1,control,2024-08-10 02:00:00,4,4,0.0,3.006557,desktop,organic,Latvia,old
1,2,control,2024-08-11 09:00:00,7,1,0.0,2.221104,mobile,organic,Latvia,old
2,3,control,2024-08-08 16:00:00,2,3,0.0,0.684445,mobile,organic,Lithuania,old
3,4,control,2024-08-09 15:00:00,4,1,0.0,23.112349,desktop,organic,Estonia,old
4,5,control,2024-08-06 19:00:00,3,1,0.0,10.166983,tablet,direct,Latvia,old
